## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

## Read-in

In [36]:
small_sites_columns = ['ADDRESS','Col1','Col2','Col3','Col4','Col5','Col6','Col7','Col8','Col9']

In [37]:
top_sites = pd.read_csv('Qatar ownership - Top sites - highlighted.csv')
maj_sites = pd.read_csv('Qatar ownership - Larger dots - maj. ownership.csv')
small_sites = pd.read_csv('Qatar ownership - Small dots - partial ownership.csv',
                          names=small_sites_columns)

## Geocode

In [38]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [39]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

In [40]:
top_sites['ADDRESS'] = top_sites['ADDRESS'] + ' Manhattan, New York City'
maj_sites['ADDRESS'] = maj_sites['ADDRESS'] + ' Manhattan, New York City'
small_sites['ADDRESS'] = small_sites['ADDRESS'] + ' Manhattan, New York City'

In [42]:
top_sites['geocoded'] = top_sites['ADDRESS'].apply(geocode)
maj_sites['geocoded'] = maj_sites['ADDRESS'].apply(geocode)
small_sites['geocoded'] = small_sites['ADDRESS'].apply(geocode)

In [43]:
top_sites['geocoded'] = top_sites['geocoded'].astype(str)
top_sites[['lat', 'lon']] = top_sites['geocoded'].str.strip('()').str.split(', ', expand=True)
top_sites['lat'] = top_sites['lat'].astype(float)
top_sites['lon'] = top_sites['lon'].astype(float)

In [44]:
maj_sites['geocoded'] = maj_sites['geocoded'].astype(str)
maj_sites[['lat', 'lon']] = maj_sites['geocoded'].str.strip('()').str.split(', ', expand=True)
maj_sites['lat'] = maj_sites['lat'].astype(float)
maj_sites['lon'] = maj_sites['lon'].astype(float)

In [45]:
small_sites['geocoded'] = small_sites['geocoded'].astype(str)
small_sites[['lat', 'lon']] = small_sites['geocoded'].str.strip('()').str.split(', ', expand=True)
small_sites['lat'] = small_sites['lat'].astype(float)
small_sites['lon'] = small_sites['lon'].astype(float)

## Set up map

In [46]:
top_sites['COLOR'] = 'orange'
maj_sites['COLOR'] = 'blue'
small_sites['COLOR'] = 'red'

In [68]:
top_sites.head(1)

,NAME,ADDRESS,$,Unnamed: 3,NOTES,geocoded,lat,lon,COLOR
0,Former Lycee Francais,"7-9 E 72 St Manhattan, New York City",$26 M,"The French style palace on East 72 Street, for...",https://nypost.com/2002/08/08/lycee-digs-now-e...,"(40.772385, -73.96611899999999)",40.772385,-73.966119,orange


In [70]:
def popup_html_top_sites(row):
    building_name = row['NAME']
    address = row['ADDRESS']
    price = row['$']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Price: </strong>{}'''.format(price) + '''<br>
    </html>
    '''
    return html

In [71]:
maj_sites.head(1)

,ADDRESS,ENTITY,AMOUNT,STATUS,NOTES,Unnamed: 5,geocoded,lat,lon,COLOR
0,"21 Beekman Place Manhattan, New York City",Private royal,$35 M,Full ownership,"June 2013, HBJ, former PM, paid $35 million fo...",Sheikh Hamad bin Jassim,"(40.75331130000001, -73.9644693)",40.753311,-73.964469,blue


In [76]:
def popup_html_top_maj_sites(row):
    address = row['ADDRESS']
    entity = row['ENTITY']
    price = row['AMOUNT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Entity: </strong>{}'''.format(entity) + '''<br>
    <strong>Price: </strong>{}'''.format(price) + '''<br>
    </html>
    '''
    return html

In [73]:
small_sites.head(1)

,ADDRESS,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,geocoded,lat,lon,COLOR
0,"640 Fifth Avenue Manhattan, New York City",Vornado portfolio,24%,https://therealdeal.com/new-york/2019/04/18/vo...,NaN,https://therealdeal.com/new-york/2019/04/18/vo...,NaN,NaN,NaN,NaN,"(40.7595199, -73.9772407)",40.75952,-73.977241,red


In [77]:
def popup_html_small_sites(row):
    address = row['ADDRESS']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [78]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=small_sites[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Qatari Sovereign Manhattan Investments')

# Create two FeatureGroups for different color pins
fg_orange = folium.FeatureGroup(name='Top Investments')
fg_blue = folium.FeatureGroup(name='Major Investments')
fg_red = folium.FeatureGroup(name='Minority Investments')

for index, row in top_sites.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        icon=folium.Icon(color=row['COLOR']),
        popup=folium.Popup(popup_html_top_sites(row), max_width=400))
    marker.add_to(fg_orange)

for index, row in maj_sites.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        icon=folium.Icon(color=row['COLOR']),
        popup=folium.Popup(popup_html_top_maj_sites(row), max_width=400))
    marker.add_to(fg_blue)
        
for index, row in small_sites.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        icon=folium.Icon(color=row['COLOR']),
        popup=folium.Popup(popup_html_small_sites(row), max_width=400))
    marker.add_to(fg_red)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)
fg_red.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m